# **Metabolic Modeling Practice Session (Google Colab Session)**

Download GEM model in EBI database (XML file): https://www.ebi.ac.uk/biomodels/MODEL1507180019#Files


NOTES:
- Other models can be downloaded in BiGG Models: http://bigg.ucsd.edu/models
- We will use KEGG for additional guidance: https://www.kegg.jp/kegg/pathway.html

## **1. Install COBRA library**

In [ ]:
!pip install cobra

## **2. Load model**

In [ ]:
# Uncomment this line to download file directly
#!wget -O MODEL1507180019_url.xml https://www.ebi.ac.uk/biomodels/model/download/MODEL1507180019?filename=MODEL1507180019_url.xml

In [ ]:
import cobra # Metabolic modeling
import pandas as pd # Handle tables

# Open XML file
model = cobra.io.read_sbml_model('MODEL1507180019_url.xml')
print(model.id)

## **3. Analyze general information**

### *3.1 Basic information*

In [ ]:
print("BASIC MODEL INFORMATION\n")
print(f"\tModel ID: {model.id}")
print(f"\tModel name: {model.name}")
print(f"\tNumber of reactions: {len(model.reactions)}")
print(f"\tNumber of metabolites: {len(model.metabolites)}")
print(f"\tNumber of genes: {len(model.genes)}")
print(f"\tNumber of compartments: {len(model.compartments)}")

### *3.2 Compartments*

In [ ]:
if model.compartments:
    print("\nCOMPARTMENTS\n")
    for comp_id, comp_name in model.compartments.items():
        print(f"\t{comp_id}: {comp_name}")

### *3.3 Metabolites*

In [ ]:
metabolites = [m for m in model.metabolites]
print("METABOLITES\n")
print(f"\tNumber of metabolites: {len(metabolites)}\n")
for metabolite in metabolites[0:3]:
    print(f"\t{metabolite.id}: {metabolite.name}")
metabolites_wout_rxn = [m for m in model.metabolites if not hasattr(m, 'formula') or not m.formula]
print(f"\n\tNumber of metabolites without reactions: {len(metabolites_wout_rxn)}")

print("\n\tExample of metadata from metabolites\n")
print(f"\t\tName: {metabolites[0].name}")
print(f"\t\tFormula: {metabolites[0].formula}")
print(f"\t\tCharge: {metabolites[0].charge}")
print(f"\t\tCompartment: {metabolites[0].compartment}")
print(f"\t\tReactions")
for reaction in metabolites[0].reactions:
    print(f"\t\t\t- {reaction}")

### *3.4 Reactions*

In [ ]:
rnxs = [r for r in model.reactions]
print("REACTIONS\n")
print(f"\tNumber of reactions: {len(rnxs)}\n")
for reaction in rnxs[0:3]:
    print(f"\tReaction ID: {reaction.id}")
    print(f"\tEnzyme: {reaction.name}")
    print(f"\tGene: {reaction.gene_reaction_rule}")
    print(f"\tReaction: {reaction.reaction}")
    print(f"\tCompartmens: {reaction.compartments}")
    print(f"\tBoundary: {reaction.boundary}")
    print(f"\tBounds: [{reaction.lower_bound:.1f}, {reaction.upper_bound:.1f}]\n")


### *3.5 Types of reactions*
- Exchange reactions (Uptake of compounds/nutrients/substrates/etc)
- Transport reactions (Move metabolites between compartments)
- Internal reactions (Biochemical transformation inside a compartment)

In [ ]:
print("REACTION ANALYSIS\n")
exchange_rxns = [r for r in model.reactions if r.boundary]
transport_rxns = [r for r in model.reactions if len(r.compartments) > 1 and not r.boundary]
internal_rxns = [r for r in model.reactions if not r.boundary and len(r.compartments) == 1]
print(f"\tNumber of exchange reactions: {len(exchange_rxns)}")
print(f"\tNumber of transport reactions: {len(transport_rxns)}")
print(f"\tNumber of internal reactions: {len(internal_rxns)}")

### *3.6 Exchange/Transport reactions*

These reactions represent the ability to take up nutrients from the medium or secrete byproducts into the medium

* Reaction ID
* _e suffix means it's in the extracellular compartment
* --> shows the reaction only goes in one direction (outside)\n"
* <=> shows a reversible reaction (in/out)

Positive Flux: Secretion (metabolite moves out of the cell/system).

Negative Flux: Uptake (metabolite moves into the cell/system).

In [ ]:
print("EXCHANGE/TRANSPORT REACTIONS (EXAMPLE 1)")
exchange_count = 0
for reaction in model.reactions:
    if exchange_count >= 3:
        break
    if reaction.boundary:
        print(f"\t- {str(reaction)}")
        print(f"\t\t  Bounds: [{reaction.lower_bound:.1f}, {reaction.upper_bound:.1f}]")
        exchange_count += 1

(In this case, yeast cannot consume these metabolites from the environment
it can only secrete it: without limitation, or exact rate is not known)

In [ ]:
print("EXCHANGE/TRANSPORT REACTIONS (EXAMPLE 2)\n")
exchange_count = 0
for reaction in model.reactions:
    if exchange_count >= 3:
        break
    if reaction.boundary:
        if reaction.lower_bound < 0:
            print(f"\t- {str(reaction)}")
            print(f"\t\t  Bounds: [{reaction.lower_bound:.1f}, {reaction.upper_bound:.1f}]")
            exchange_count += 1

(In this case, D-glucose is taken at 1.0 mmol/gDW/h, and it can be freely secrete it from the system. Both H20 and NH4 moves through cell through passive diffusion)


### *3.7 Internal reactions*

Reversible (<=>) or forward (-->)

In [ ]:
print("Reversible reactions\n")
exchange_count = 0
for reaction in model.reactions:
    if exchange_count >= 5:
        break
    if reaction.lower_bound < 0 and reaction.upper_bound > 0:
        print(f"\t- {str(reaction)}")
        print(f"\t\t  Bounds: [{reaction.lower_bound:.1f}, {reaction.upper_bound:.1f}]")
        exchange_count += 1

In [ ]:
print("Forward reactions\n")
exchange_count = 0
for reaction in model.reactions:
    if exchange_count >= 5:
        break
    if reaction.upper_bound > 0 and reaction.lower_bound == 0:
        print(f"\t- {str(reaction)}")
        print(f"\t\t  Bounds: [{reaction.lower_bound:.1f}, {reaction.upper_bound:.1f}]")
        exchange_count += 1

### _3.8 Zero bound reactions_

This reactions are blocked. We can modify boundaries to simulate gene knockouts.

In [ ]:
reactions_zero_bounds = [r for r in model.reactions if r.lower_bound == 0 and r.upper_bound == 0]
print(f"Reactions with [0,0] bounds: {len(reactions_zero_bounds):,}")

### *3.9 Gene-reaction connectivity*

In [ ]:
print("GENE-REACTION CONNECTIVITY\n")
print(f"\tNumber of genes: {len(model.genes)}")
print(f"\tNumber of reactions: {len(model.reactions)}")
rxns_with_grc = [r for r in model.reactions if r.gene_reaction_rule]
print(f"\tNumber of reactions with GRC: {len(rxns_with_grc)}")
rxns_wout_grc = [r for r in model.reactions if not r.gene_reaction_rule]
print(f"\tNumber of reactions without GRC: {len(rxns_wout_grc)}")

### *3.10 Objective function*

The objective function is a mathematical expression that guides the optimization process to find the most likely metabolic state of an organism. The most common objective function is the biomass objective function (BOF), which represents the cell's growth by maximizing the rate of production of all necessary biomass components like DNA, RNA, and proteins.

In [ ]:
print("OBJECTIVE FUNCTIONS\n")
objective_reactions = []
for reaction in model.reactions:
    if reaction.objective_coefficient != 0:
        objective_reactions.append((reaction, reaction.id, reaction.objective_coefficient))

if objective_reactions:
    for rxn, rxn_id, coeff in objective_reactions:
        print(f"\t- {rxn_id}: {coeff}")
        print(f"\t- Reaction: {str(rxn)}")
else:
    print("No objective functions")

(By defining this objective, models can use techniques like Flux Balance Analysis (FBA) to calculate the flux (rate) through all metabolic reactions in the network, predicting phenotypes like growth rate.)

## **4. Project example**

Modify *S. cerevisiae* to produce monoterpenes using synthetic biology and optimizing through metabolic engineering. We will use glucose as our substrate.

Monoterpene biosynthesis: https://www.kegg.jp/pathway/map00902

### *Work with tables dynamically*

In [ ]:
!pip install itables

In [ ]:
from itables import init_notebook_mode, show
import pandas as pd

# Initialize itables for interactive display
init_notebook_mode(all_interactive=True)

### *Identify metabolite IDs*

Glucose and Geranyl-PP

In [ ]:
# Convert metabolite metadata as a DataFrame
metabolites_data = []
for met in model.metabolites:
    metabolites_data.append({
        'ID': met.id,
        'Name': met.name,
        'Formula': met.formula,
        'Charge': met.charge,
        'Compartment': met.compartment
    })
df = pd.DataFrame(metabolites_data)
df

Geranyl diphosphate: grdp_c

D-Glucose: glc_DASH_D_e

## **5. Simulate initial production**

- Minimal medium (Glucose-only)

Steps:

1. Knock transport of other extracellular compounds inside the cell.
2. Set the substrate as the carbon source.
3. Create a demand reaction (a demand reaction is an artificial, irreversible reaction that represents the cellular "demand" for specific metabolites).
4. Set demand reaction as a objective function to maximize the flux through the new demand reaction.
5. Run Flux Balance Analysis (FBA).
6. Print results (Reduced costs and shadow price).

In [ ]:
# Uncomment this line to (re-)run the process
model = cobra.io.read_sbml_model('MODEL1507180019_url.xml')

In [ ]:
# Set IDs
glc_id = 'glc_DASH_D_e' # Substrate
grdp_id = 'grdp_c' # Product

In [ ]:
# Knock uptake of other extracellular compounds
knocked_rxn = 0
for reaction in model.reactions:
    if reaction.boundary:
        reaction.lower_bound = 0.0
        reaction.upper_bound = 0.0
        knocked_rxn += 1
print(f"Reactions knocked: {knocked_rxn}")

In [ ]:
# Set glucose as the carbon source (set uptake rate)
for rxn in model.metabolites.get_by_id("glc_DASH_D_e").reactions:
    # Find the reaction that uptakes glucose inside
    if rxn.boundary:
        rxn.lower_bound = -10.0
        rxn.upper_bound = 0.0
        print("Modified reaction:\n")
        print(f"\tReaction ID: {rxn.id}")
        print(f"\tReaction: {str(rxn.reaction)}")
        print(f"\tBounds: [{rxn.lower_bound:.1f}, {rxn.upper_bound:.1f}]\n")

In [ ]:
# Get the target metabolite object
# Ensure the correct reactions are found in model
target_metabolite = model.metabolites.get_by_id(grdp_id)
for rxn in target_metabolite.reactions:
    print(f"\tReaction ID: {rxn.id}")
    print(f"\tReaction: {str(rxn.reaction)}")

In [ ]:
# 1. Set the Demand Reaction Object
DEMAND_REACTION_ID = f"DM_{grdp_id}"
demand_reaction = cobra.Reaction(DEMAND_REACTION_ID)
demand_reaction.name = f"Demand reaction for {grdp_id}"

# 2. Set stoichiometry: -1.0, let it be produced by the model
demand_reaction.add_metabolites({target_metabolite: -1.0})

# 3. Set the reaction bounds
# (Forward reaction only, for production)
demand_reaction.lower_bound = 0.0
demand_reaction.upper_bound = 1000.0

# 4. Add the reaction to the model
model.add_reactions([demand_reaction])

print("Modified demand reaction:\n")
print(f"\tReaction ID: {demand_reaction.id}")
print(f"\tReaction: {str(demand_reaction.reaction)}")

In [ ]:
# Print current Objective function
current_objective = model.objective
print(f"Biomass reaction ID: {current_objective}")

In [ ]:
# Set Demand function as Objective function
model.objective = DEMAND_REACTION_ID
print(f"Set {DEMAND_REACTION_ID} as Objective function")
print(f"Current Objective Expression: {model.objective.expression}")

In [ ]:
# Get Biomass reaction
BIOMASS_REACTION_ID = ""
for rxn in model.reactions:
    if "biomass" in rxn.id:
        BIOMASS_REACTION_ID = rxn.id
        print(f"\tReaction ID: {rxn.id}")
        print(f"\tReaction: {str(rxn.reaction)}")
        print(f"\tBounds: [{rxn.lower_bound:.1f}, {rxn.upper_bound:.1f}]\n")

In [ ]:
# Run Flux Balance Analysis (FBA)
solution = model.optimize()

Analyze if model is optimal or infeasible

In [ ]:
# Print Results
print("--- FBA Optimization Results ---")
print(f"Status: {solution.status}")

if solution.status == 'optimal':
    max_production = solution.fluxes[DEMAND_REACTION_ID]
    print(f"Maximum Geranyl-PP Production: {max_production:.4f} mmol/gDW/h")

    # Check associated growth rate
    if BIOMASS_REACTION_ID in solution.fluxes:
        growth_rate = solution.fluxes[BIOMASS_REACTION_ID]
        print(f"Growth Rate: {growth_rate:.4f} 1/h")

Notes:

- A Glucose-estricted medium will not allow *S. cerevisiae* to produce Geranyl-PP or even grow.
- Growth-rate at 0.0000 1/h

In [ ]:
# Modify lower bound of biomass
# (10% of the maximum theoretical growth)
model.reactions.get_by_id(BIOMASS_REACTION_ID).lower_bound = 0.1

In [ ]:
# Run Flux Balance Analysis (FBA)
solution = model.optimize()

# Print Results
print("--- FBA Optimization Results ---")
print(f"Status: {solution.status}")

if solution.status == 'optimal':
    max_production = solution.fluxes[DEMAND_REACTION_ID]
    print(f"Maximum Geranyl-PP Production: {max_production:.4f} mmol/gDW/h")

    # Check associated growth rate
    if BIOMASS_REACTION_ID in solution.fluxes:
        growth_rate = solution.fluxes[BIOMASS_REACTION_ID]
        print(f"Growth Rate: {growth_rate:.4f} 1/h")

In [ ]:
model.metabolites.grdp_c.summary()

*Analysis 1: Key reaction fluxes*

- Data: fluxes, reduced_costs

Fluxes column interpretation:

- Positive value: Reaction proceeds in the forward direction.
- Negative value: Reaction proceeds in the reverse direction
- Zero: Reaction not active.

Reduced_costs column interpretation:

- Zero: Reaction is not constraining the optimal solution.
- Negative value: Reaction is at its upper bound. Increasing this bound would improve the objective.
- Positive value: Reaction is at its lower bound. Decreasing this bound would improve the objective.

In [ ]:
flux_df = solution.to_frame()
flux_df

In [ ]:
# Identify reactions at highest RC in upper bound
# (Increase to improve Objective)
highest_rc = flux_df.loc[flux_df['reduced_costs'] > 0.0].sort_values(by='reduced_costs', ascending=False).head(5)
print(highest_rc)
# Get the names of enzymes and reaction
for index, row in highest_rc.iterrows():
    # Get column name
    reaction_id = index
    reaction = model.reactions.get_by_id(reaction_id)
    print("\n")
    print(f"Reaction ID: {reaction_id}")
    print(f"Enzyme: {reaction.name}")
    print(f"Reaction: {reaction.reaction}")
    print(f"Bounds: [{reaction.lower_bound:.1f}, {reaction.upper_bound:.1f}]")

In [ ]:
# Identify reaction at lowest RC in lower bound
# (decrease to improve)
lowest_rc = flux_df.loc[flux_df['reduced_costs'] < 0.0].sort_values(by='reduced_costs', ascending=True).head(5)
print(lowest_rc)
# Get the names of enzymes and reaction
for index, row in lowest_rc.iterrows():
    # Get column name
    reaction_id = index
    reaction = model.reactions.get_by_id(reaction_id)
    print("\n")
    print(f"Reaction ID: {reaction_id}")
    print(f"Enzyme: {reaction.name}")
    print(f"Reaction: {reaction.reaction}")
    print(f"Bounds: [{reaction.lower_bound:.1f}, {reaction.upper_bound:.1f}]")

*Analysis 2: Shadow Price*

Identify precursor that influence Objective function.

- Positive value: Greater concentration of precursor would improve the objective.
- Negative value: Lower concentration of precursor would improve the objective.

In [ ]:
shadow_price_df = solution.shadow_prices.to_frame(name='shadow_price')
shadow_price_df

In [ ]:
# Identify metabolites with higher influence
high_shadow = shadow_price_df.loc[shadow_price_df['shadow_price'] > 0.0].sort_values(by='shadow_price', ascending=False).head(5)
print(high_shadow)
# Get the metabolites name
for index, row in high_shadow.iterrows():
    # Get column name
    metabolite_id = index
    metabolite = model.metabolites.get_by_id(metabolite_id)
    print("\n")
    print(f"Metabolite ID: {metabolite_id}")
    print(f"Name: {metabolite.name}")

In [ ]:
# Identify metabolites with lower influence
low_shadow = shadow_price_df.loc[shadow_price_df['shadow_price'] < 0.0].sort_values(by='shadow_price', ascending=True).head(5)
print(low_shadow)
# Get the metabolites name
for index, row in low_shadow.iterrows():
    # Get column name
    metabolite_id = index
    metabolite = model.metabolites.get_by_id(metabolite_id)
    print("\n")
    print(f"Metabolite ID: {metabolite_id}")
    print(f"Name: {metabolite.name}")

## **5. Synthetic Biology, Metabolic Engineering and Gene Knockouts**

We will need to get a list of:

- Main reactions
- Metabolic leak reactions

## **Self-notes (Mau)**

- Create a script to convert KEGG IDs to IDs from GEM model.
- Create a script to print KEGG-like images to visualize the results of optimized model.